<a href="https://colab.research.google.com/github/naru289/Assignment-16/blob/main/M3_AST_16_Text_Representation_%26_Deep_Averaging_Network_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment : Natural Language Processing - I (Text representation and Deep Averaging Network)

### Learning Objectives:

At the end of the experiment, you will be able to:
 
*  generate vector representation of words in the data using Glove embeddings
*  find the cosine similarity between the words present in the data
*  plot the word vector representation using TSNE
*  implement Deep Averaging Network 

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()
  
notebook= "M3_AST_16_Text_Representation_&_Deep_Averaging_Network_C" #name of the notebook

def setup():
    ipython.magic("sx wget -qq https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/glove.6B.zip")
    ipython.magic("sx unzip glove.6B.zip")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/rt-polarity.zip")
    ipython.magic("sx unzip rt-polarity.zip")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None
    
    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None
  
  
# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None
  
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None
  

def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError 
    else: 
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError 
    else: 
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None
  

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup() 
else:
  print ("Please complete Id and Password cells before running setup")



### Dataset Description

The **sentence polarity dataset v1.0** contains two data files which are: 
  * **rt-polarity.pos**: It contains 5331 positive examples
  * **rt-polarity.neg**: It contains 5331 negative examples

Each line in these two files corresponds to a single snippet (usually
containing roughly one single sentence) that includes the review of a movie. 

**Note:** Here is the source [link](https://www.cs.cornell.edu/people/pabo/movie-review-data/) to the Movie  dataset





### Introduction

The aim of this assignment is to study the use of pre-trained word embeddings using GloVe for text representation. The pre-trained word embeddings of different dimensions are provided. A simple way to represent a piece of text is to tokenize, vectorize and average the word vector representations of all the words in
the text. You will study the use of this simple approach of text representation.

1. We will use t-SNE to plot the word vector representations.
Indicate different classes with different colors. Also, we have used 50-dimensional word embeddings in this experiment. *If you want to study the use of different dimensional word (100d,200,or 300d), just change the dimensions and use the embeddings.*
2. Design a classifier using Deep Averaging Network (DAN) on datasets provided and report the test set accuracy.

### Importing the libraries and packages

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
import itertools
import seaborn as sns
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.patches as mpatches
tsne = TSNE(n_components=2)
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision
import torchvision.transforms as transforms
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity

### Loading the data

In [ ]:
# Read the positive and negative files and split the sentences into a list
with open('rt-polarity.neg',"r") as data_neg:
  data_neg_set = data_neg.read().splitlines()

with open('rt-polarity.pos',"r") as data_pos:
  data_pos_set = data_pos.read().splitlines()

In [ ]:
# Length of the positive and negative reviews
len(data_neg_set), len(data_pos_set)

In [ ]:
# Loading the negative reviews
data_neg_set = pd.DataFrame(data_neg_set, columns=["Review"])

# Loading the positive reviews
data_pos_set = pd.DataFrame(data_pos_set, columns=["Review"])

In [ ]:
# Print the first five rows of the positive examples
data_pos_set.head()

In [ ]:
# Print the first five rows of the negative examples
data_neg_set.head()

#### Giving the labels to the data

Let us give the labels as positive and negative for the sentences present in the two files.

In [ ]:
data_neg_set['Label'] = 'Negative'
data_pos_set['Label'] = 'Positive'

Let us have a glance at few of the values present in the data with negative and positive reviews that we have labeled in the previous step.

In [ ]:
data_neg_set.head()

In [ ]:
data_pos_set.head()

#### Combining the positive and negative data

Now, we have to work on the combined data containing the positive and negative reviews, so, let us concatenate both the dataframes. 

In [ ]:
dataframes = [data_neg_set, data_pos_set]
rt_polarity_data = pd.concat(dataframes)
rt_polarity_data


From above we can see that due to concatenation the last row index is 5330 that we can see below but that should be 10661.

Therefore, we will reset the index so that last row has index of 10661.

In [ ]:
rt_polarity_data.reset_index(drop = True, inplace = True) 

In [ ]:
rt_polarity_data.tail()

When you combine the negative and positive examples, it is a good idea to shuffle the examples so that the negative and positive examples are spread throughout. If we do not shuffle it, then, it may happen that in some mini-batches, examples from only one class(positive or negative) will be present. Therefore, it is better to avoid such scenarios.


In [ ]:
rt_polarity_data = shuffle(rt_polarity_data)

Now, let us look at the shuffled data.

In [ ]:
rt_polarity_data

Let us check the value counts of negative and positive reviews.

In [ ]:
rt_polarity_data['Label'].value_counts()

Checking whether there are any null values present in the data.

In [ ]:
rt_polarity_data.isnull().values.any()

From above we can see that there are no null values present in the data.

### Data Preprocessing 

In this step, we will preprocess the data to remove non-alphabet (punctuations and numbers), stop words,  and lower case all of the reviews present in the data.

In [ ]:
def preprocess_text(sen):
    
    sen = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", sen)     
    sen = re.sub(r"\'s", " \'s", sen) 
    sen = re.sub(r"[\([{})\]]", "", sen)

    # Tokenizing words
    tokens = word_tokenize(sen)  

    # Converting to lower case
    tokens = [w.lower() for w in tokens]    

     # Remove punctuations
    table = str.maketrans('', '', string.punctuation) 
    stripped = [w.translate(table) for w in tokens]

    # Remove non alphabet
    words = [word for word in stripped if word.isalpha()]  
    stop_words = set(stopwords.words('english'))

    # Remove stop words
    words = [w for w in words if not w in stop_words]  
    
    return words

In [ ]:
# Store the preprocessed reviews in a new list
lines_pos = []
sentences_pos = list(data_pos_set['Review'])

In [ ]:
for sen in sentences_pos:
    # Call the preprocess_text function on each sentence of the review text 
    lines_pos.append(preprocess_text(sen))

In [ ]:
# Check for the length of the preprocessed text
len(lines_pos)

In [ ]:
# Print the preprocessed text for the first review
print(lines_pos[0])

Further, let us perform the data preprocessing for negative reviews.


In [ ]:
# Store the preprocessed reviews in a new list
lines_neg = []
sentences_neg = list(data_neg_set['Review'])

In [ ]:
for sen in sentences_neg:
  # Call the preprocess_text function on each sentence of the review text 
  lines_neg.append(preprocess_text(sen))    

In [ ]:
# Check for the length of the preprocessed text
len(lines_neg)

In [ ]:
# Print the preprocessed text for the first review
print(lines_neg[0])

Now, as we can see that lines_neg and lines_pos are the lists of list. So, we have to convert it into a simple list so that we can use the elements(words) of the list to generate vectors. 

In [ ]:
text_neg = list(itertools.chain.from_iterable(lines_neg))
text_pos = list(itertools.chain.from_iterable(lines_pos))

Let us look into the first 10 values in text_neg and text_pos and the length of both the lists.

In [ ]:
text_neg[0:9]

In [ ]:
text_pos[0:9]

In [ ]:
len(text_neg)

There are 55688 words for negative reviews.

In [ ]:
len(text_pos)

There are 56832 words for positive reviews.

### **Load the GloVe embeddings**

**What is GloVe?**

GloVe stands for global vectors for word representation. It is an unsupervised learning algorithm developed by Stanford for generating word embeddings by aggregating global word-word co-occurrence matrix from a corpus. Word embeddings are basically a form of word representation that bridges the human understanding of language to that of a machine. Meaning that two similar words are represented by almost similar vectors that are very closely placed in a vector space. These are essential for solving most Natural language processing problems.The resulting embeddings show interesting linear substructures of the word in vector space.

Thus when using word embeddings, all individual words are represented as real-valued vectors in a predefined vector space. Each word is mapped to one vector and the vector values are learned in a way that resembles a neural network.

Now, let us load the 50-dimensional GloVe embeddings. But if you want to use 100d, 200d, or 300d embeddings, then, just change the dimensions in the name of the model.

In [ ]:
GloVe_Dict_50d = {}
# Loading the 50-dimensional vector of the model
with open("glove.6B.50d.txt", 'r') as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      GloVe_Dict_50d[word] = vector

Let us have a look at the Glove_Dict.

In [ ]:
GloVe_Dict_50d

Above, we can see that each word is represented by 50 values.

In [ ]:
# Length of the word vocabulary
print(len(GloVe_Dict_50d))

From above, we can see that the glove dictionary consist of 400000 words. Further, let us create vector representation of the words for negative and positive reviews using the glove dictionary.

#### Vector representation of words

In [ ]:
def gen_vectors(text):
  vectors = []
  for word in text:
    try:
      vector = GloVe_Dict_50d[word]
      vectors.append(vector)
    except KeyError:
      pass
  print("There are %d words and the vector size of each word is %d" %((len(vectors),len(vectors[0]))))
  return vectors

In [ ]:
# Passing the words present in text_neg and text_pos to the function gen_vectors
vectors_neg = gen_vectors(text_neg)
vectors_pos = gen_vectors(text_pos)


### Find the similarity between words 

Here, we will measure the similarity between the words using cosine_similarity. Now, let us first understand about **cosine similarity**.

**cosine similarity** : Cosine similarity is one of the metric to measure the text-similarity between two documents irrespective of their size in Natural language Processing. If the Cosine similarity score is 1, it means two vectors have the same orientation and they are closely similar. The value closer to 0 indicates that the two words have less similarity.


First, we will find the cosine similarity between the words present in the negative reviews(text_neg).

In [ ]:
# Creating an empty list for storing values of cosine similarity
word_similarity = []
index = []
for i, word_1 in enumerate(text_neg):
  row_wise_simiarity = []
  print(i,word_1)
  if(i==4):
    break
  for j, word_2 in enumerate(text_neg):
    # Get the vectors of the word using GloVe
    try:
      vec_1, vec_2 = GloVe_Dict_50d[word_1], GloVe_Dict_50d[word_2]
    except KeyError:
      pass

    # As the vectors are in one dimensional, convert it to 2D by reshaping
    vec_1, vec_2 = np.array(vec_1).reshape(1,-1), np.array(vec_2).reshape(1,-1) 

    # Measure the cosine similarity between the vectors.
    similarity = cosine_similarity(vec_1, vec_2)
    row_wise_simiarity.append(np.array(similarity).item())

  # Store the cosine similarity values in a list  
  word_similarity.append(row_wise_simiarity)
  index.append(word_1)

# Create a DataFrame to view the similarity between words
pd.DataFrame(word_similarity, columns=text_neg,index = index)

From the above similarity scores, we can observe that words which are closely similar have values close to 1 and which are non-similar are close to 0. For example, **simplistic** and **silly** has score 0.68, that means they can be considered as 68% similar.

Now, we will find the cosine similarity between the words present in the positive reviews(text_pos).

In [ ]:
word_similarity = []
index = []
for i, word_1 in enumerate(text_pos):
  row_wise_simiarity = []
  print(i,word_1)
  if(i == 4):
    break
  for j, word_2 in enumerate(text_pos):
    # Get the vectors of the word using GloVe
    try:
      vec_1, vec_2 = GloVe_Dict_50d[word_1], GloVe_Dict_50d[word_2]
    except KeyError:
      pass

    # As the vectors are in one dimensional, convert it to 2D by reshaping
    vec_1, vec_2 = np.array(vec_1).reshape(1,-1), np.array(vec_2).reshape(1,-1) 

    # Measure the cosine similarity between the vectors.
    similarity = cosine_similarity(vec_1, vec_2)
    row_wise_simiarity.append(np.array(similarity).item())

  # Store the cosine similarity values in a list  
  word_similarity.append(row_wise_simiarity)
  index.append(word_1)

# Create a DataFrame to view the similarity between words
pd.DataFrame(word_similarity, columns=text_pos, index = index)

Again, from the above scores we can observe the similarity of words present in the list of words in positive reviews.

### Visualization of word vectors using TSNE

To represent the word vectors we will vectorize the **"Review"** column of **rt_polarity_data** and get the average of the vectors using the 50-dimensional GloVe embeddings.

**How t-SNE works?**

The intuition of what and how t-SNE works.

Suppose you have a 50-dimensional data set, as it is like an impossible task for us to visualize and get a sense of it. We have to convert that 50D data set to something which we can visualize or with which we can play around. This is where t-SNE comes into the picture it converts the higher dimensional data into the lower dimensional data by following steps:-

1. It measures the similarity between the two data points and it does for every pair. Similar data points will have more value of similarity and the different data points will have less value.

2. Then it converts that similarity distance to probability(joint probability) according to the normal distribution.

3. It does the similarity check for every point. Thus it will have the similarity matrix `S1` for every point. This is all calculation it does for our data points that lie in higher-dimensional space.

4. Now, t-SNE arranges all of the data points randomly on the required lower-dimensional.

5. And it does all of the same calculation for lower dimensional data points as it does for higher ones — calculating similarity distance but with a major difference it assigns probability according to t- distribution instead of normal distribution and this is because it is called t-SNE not simple SNE.

6. Now we also have the similarity matrix for lower dimensional data points. Let’s call it S2.

7. Now, what t-SNE does is it compares matrix S1 and S2 and tries to make the difference between matrix S1 and S2 much smaller by doing some complex mathematics.

In the end, we will have lower-dimensional data points that try to capture even complex relationships at which PCA fails.
So on a very high level, this is how t-SNE works.

For this, we will define a function that will help us to get the glove embeddings.

In [ ]:
def glove_embeddings(text, dim):
    if len(text) < 1:
        return np.zeros(dim)
    else:
        vectorized = [GloVe_Dict_50d[word] if word in GloVe_Dict_50d else np.random.randn(dim) for word in text]  
    sum = np.sum(vectorized, axis=0)
    # Return the average vector
    return sum/len(vectorized)     

def get_glove_embeddings(text, dimension):
        embeddings = text.apply(lambda x: glove_embeddings(x, dimension))
        return list(embeddings)

Getting the embeddings for 'Review' column of the rt-polarity data.

In [ ]:
word_embeddings = get_glove_embeddings(rt_polarity_data['Review'], dimension=50)

Now, let us visualize the positive and negative reviews associated with the word embeddings.  

In [ ]:
def tsne_visualization(word_embeddings):
    x = word_embeddings[1:100]
    x = np.asarray(x)
    y = tsne.fit_transform(x)
    plt.figure(figsize=(20,10))
    colors=['orange','red']
    sns.scatterplot(x=y[:,0],y=y[:,1],hue=rt_polarity_data['Label'].iloc[1:100])

    for label,x,y in zip(rt_polarity_data['Label'].iloc[1:100],y[:, 0],y[:,1]):
        plt.annotate(label,xy=(x,y),xytext=(0,0),textcoords='offset points')
    plt.show()

In [ ]:
tsne_visualization(word_embeddings)

Further, let us move on to the next topic which is Deep Averaging Network.

### Deep Averaging Network

Vector space models for natural language processing (NLP) represent words using low dimensional vectors called embeddings. To apply vector space
models to sentences or documents, one must first select an appropriate composition function, which is a mathematical process for combining multiple
words into a single vector. Composition functions fall into two classes: **unordered** and **syntactic**. **Unordered functions** treat input texts as bags of word embeddings, while **syntactic functions** take word order and sentence structure into account. 

Previously published experimental results have shown that syntactic functions outperform unordered functions on many tasks. However, there is a tradeoff: syntactic functions
require more training time than unordered composition functions and are prohibitively expensive in
the case of huge datasets or limited computing resources. 

Therefore, we introduce a deep unordered model that obtains near state-of-the-art accuracies on a variety of sentence and document-level tasks with just minutes of training time on an average laptop computer. This model, the deep averaging network (DAN), works in three simple steps:

1. Take the vector average of the embeddings
associated with an input sequence of tokens.
2. Pass that average through one or more feedforward layers.
3. Perform (linear) classification on the final
layer’s representation.

The model can be improved by applying a novel dropout-inspired regularizer: for each training instance, randomly drop some of the tokens’ embeddings before computing the average.

Also, let us look into the architecture of Deep Averaging Network that we will create.

![image.png](https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/DNN.png)



In the above architecture, we pass the average of word vectors into the layers of neural network to train the model and perform the classification at the final layer.

Now, let us do some preprocessing before craeting the Deep Averaging Network. 
 

We can preprocess the text using gensim package. Gensim provides function **simple_preprocess** for more effective preprocessing of the corpus. In such kind of preprocessing, we can convert a document into a list of lowercase tokens. We can also ignore tokens that are too short or too long.

**Note:** Refer to the following [link](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess) for gensim `simple_preprocess` method

In [ ]:
stop_words = set(stopwords.words('english'))

rt_polarity_data['Review'] = rt_polarity_data['Review'].apply(lambda x:simple_preprocess(x, max_len=40))

# Remove stop words
rt_polarity_data['Review'] = rt_polarity_data['Review'].apply(lambda x: [w for w in x if not w in stop_words])

In [ ]:
rt_polarity_data.head()

Further, we will replace the positive label with '1' and negative with '0'.

In [ ]:
rt_polarity_data["Label"] = rt_polarity_data["Label"].apply(lambda x:1 if x == "Positive" else 0)

In [ ]:
rt_polarity_data

Storing the 'Review' and the 'Label' column in X and y.

In [ ]:
X = rt_polarity_data["Review"]
y = rt_polarity_data['Label']

Getting the glove embeddings for 'Review' column of the rt-polarity data.

In [ ]:
train_embeddings  = get_glove_embeddings(rt_polarity_data['Review'], dimension=50)

In [ ]:
len(train_embeddings[0])

Let us now prepare the data for train and test.

In [ ]:
# Storing the train_embeddings in X
X = np.array(train_embeddings)

# Converting X into torch tensor
X = torch.Tensor(X)

# Reshaping X to 200 dimension
X = X.reshape(-1, 50)
print(X.shape)

In [ ]:
# Storing the labels in y
y = rt_polarity_data['Label']

# Converting X into torch tensor
y = torch.Tensor(y)

# Reshaping y to 1 dimension
y = y.reshape(-1,1)
print(y.shape)

Here, we will split the data into train and test part with test size as 0.15 equivalent to 15% of the data using stratified sampling. Moreover,  stratified sample is one that ensures that subgroups (strata) of a given population are each adequately represented within the whole sample population of a research study.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 10000)

In [ ]:
# Set up device to run CUDA operations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Preparing the training data and testing data with batch size 64 using Tensor Dataset and Tensor Data Loader.

In [ ]:
train_dataset = TensorDataset(X_train,y_train)
train_loader = DataLoader(train_dataset,batch_size = 64)
test_dataset = TensorDataset(X_test,y_test)
test_loader = DataLoader(test_dataset,batch_size = 64)

#### Building the DAN model

In this Deep Averaging Network model, we using 3 layers with dropout as 50% and batch normalization.

In [ ]:
class DAN(torch.nn.Module):
        def __init__(self, input_size, hidden_size, dp = 0.5, d_out = 2):
            super(DAN, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.bn1 = nn.BatchNorm1d(input_size)
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.dropout1 = nn.Dropout(dp)
            self.bn2 = nn.BatchNorm1d(self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, 10)
            self.fc3 = torch.nn.Linear(10, d_out)         
            
        def forward(self, x):
            # x = self.dropout1(x)
            x = self.bn1(x)
            x = self.fc1(x)
            x = self.dropout1(x)
            x = self.bn2(x)
            x = self.fc2(x)
            x = self.fc3(x)
            return x

#### Model, Criterion, and Optimizer

**Cross Entropy Loss** : Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label. So predicting a probability of .012 when the actual observation label is 1 would be bad and result in a high loss value. A perfect model would have a log loss of 0.

**Adam Optimizer** : Adam is a replacement optimization algorithm for stochastic gradient descent for training deep learning models. Adam combines the best properties of the AdaGrad and RMSProp algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.

In [ ]:
# Dimension as 50 and layers as 32 
model = DAN(50, 25)  
criterion = nn.CrossEntropyLoss()
# We will set the learning rate (lr) as 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

Let us have a glance at the structure of model that we have built.

In [ ]:
print(model)

#### Training the model

In [ ]:
# First switch the module mode to model.train() so that new weights can be learned after every epoch. 
model.train()

# No of Epochs
epochs = 50

for epoch in range(epochs):

  # Iterate through all the batches in each epoch
  for inputs, target in train_loader:

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)

    # Compute Loss
    target = target.squeeze_()
    target = target.type(torch.LongTensor)

    loss = criterion(outputs, target)
   
  print('Epoch {}: train loss: {}'.format(epoch, loss.item()))

  # Backward pass
  loss.backward()
  
  # optimizer.step() updates the weights accordingly
  optimizer.step()

print("We got the training loss as %f for %d epochs" %((loss.item(), epochs)))

#### Model evaluation for test set

In [ ]:
# Creating empty lists to store the labels and the predictions
labels = []
predictions = []

In [ ]:
model.eval()

for inputs,target in test_loader:
    
    # Forward pass
    outputs = model(inputs)

    _,out = torch.max(outputs, 1)

    labels.append(out)

    target = target.squeeze_()
    target = target.type(torch.LongTensor)

    predictions.append(target)
    loss = criterion(outputs,target)

print("We got the loss as %f for test set." %((loss.item())))

Further, let us check how accurate is the model by checking how many labels matches the predictions.

In [ ]:
labels = torch.cat(labels, 0)
predictions = torch.cat(predictions,0)

In [ ]:
j = 0
for i in range(662):
  if labels[i] == predictions[i]:
    j+=1
print("%d predicted values matches the label out of total %d training set values." %((j,len(test_dataset))))

### Please answer the questions below to complete the experiment:




**Statement** : Deep unordered model that obtains near state of art accuracy on sentence and document level tasks with very less training time works in the following steps:

(a) take the vector average of the embeddings associated with an input sequence of tokens

(b) pass that average through one or more feed-forward layer

(c) perform (linear) classification on the final layers representation

(d) Mean squared error.

(e) Loss function is cross entropy.

In [ ]:
#@title Q.1. Which of the following steps are true for the above statement? 
Answer1 = "" #@param ["","a, b, and c","a and b","a, b, c, and d","a, b, c, and e","All of the above"]


**Consider the following statements for GloVe and answer Question 2**

A. GloVe creates a global co-occurrence matrix by estimating the probability a given word will co-occur with other words. 

B.  GloVe embeddings are learnt based on matrix factorization techniques.

C.  GloVe does not use neural networks.

D.   GloVe embeddings are based on training a shallow feedforward neural network.

In [ ]:
#@title Q.2. Which of the above statement(s) is TRUE for GloVe?
Answer2 = "" #@param ["","A and B","B and C","A, B, and C ","A and D","only D"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")